In [1]:
import os
import sys
from dask.distributed import Client
# client = Client(scheduler_file='/proj/kimyy/Dropbox/source/python/all/mpi/scheduler.json', threads_per_worker=2, n_workers=6)
client = Client(scheduler_file='/proj/kimyy/Dropbox/source/python/all/mpi/scheduler.json')  
# client = Client(scheduler_file='/proj/kimyy/Dropbox/source/python/all/mpi/scheduler_10.json')  

# add private module path for workers
# client.run(lambda: os.environ.update({'PYTHONPATH': '/proj/kimyy/Dropbox/source/python/all/Modules/CESM2'}))
# def add_path():
#     if '/proj/kimyy/Dropbox/source/python/all/Modules/CESM2' not in sys.path:
#         sys.path.append('/proj/kimyy/Dropbox/source/python/all/Modules/CESM2')

# client.run(add_path)

def setup_module_path():
    module_path = '/proj/kimyy/Dropbox/source/python/all/Modules/CESM2'
    if module_path not in sys.path:
        sys.path.append(module_path)

client.run(setup_module_path)

client

<Client: 'tcp://203.247.189.225:42059' processes=5 threads=90, memory=419.10 GiB>

In [2]:
# load public modules

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy import stats
import cmocean
from cmcrameri import cm
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import cftime
import pop_tools
from pprint import pprint
import time
import subprocess
import re as re_mod
import cftime
import datetime
from scipy.stats import ttest_1samp
import xcesm

In [3]:
# load private modules

import sys
sys.path.append('/proj/kimyy/Dropbox/source/python/all/Modules/CESM2')
from KYY_CESM2_preprocessing import CESM2_config

In [4]:
cfg_var_TEMP=CESM2_config()
cfg_var_TEMP.year_s=1850
cfg_var_TEMP.year_e=2026
# cfg_var_TEMP.year_e=1861

cfg_var_TEMP.setvar('TEMP')

if cfg_var_TEMP.comp=='ocn':
    ds_grid = pop_tools.get_grid('POP_gx1v7')

#thermocline depth anomalies averaged over the equatorial Pacific 120° E–80° W, 5° S–5° N (h), that is, the
# WWV index (with a constant factor of the area it covers).


In [5]:
# define preprocessing function


# weighted_avg_manual = xr.Dataset()

exceptcv=['time','lon','lat','lev', 'TLONG', 'TLAT', 'z_w', cfg_var_TEMP.var]


def process_coords_sst(varname, ds, sd, ed, drop=True, except_coord_vars=['time','lon','lat','lev', 'ilev', 'AREA', 'SST']):
    """Preprocessor function to drop all non-dim coords, which slows down concatenation."""
    coord_vars = []
    for v in np.array(ds.coords) :
        if not v in except_coord_vars:
            coord_vars += [v]
    for v in np.array(ds.data_vars) :
        if not v in except_coord_vars:
            coord_vars += [v]

    # ds = ds.where(mask, drop=True)
    # ds= ds.isel(z_t=slice(0, 31))
    
    # volume= ds_grid_sub['dz'] * ds_grid_sub['TAREA']
    # weighted_sum = (ds[varname] * volume).sum(dim=['lat','lon'], skipna=True)
    # effective_volume = volume.where(ds[varname].notnull()).sum(dim=['lat','lon'], skipna=True)
    # ds[varname] = weighted_sum / effective_volume
    
    
    if drop:
        ds= ds.drop(coord_vars)
        ds= ds.sel(time=slice(sd, ed))
        return ds
    else:
        return ds.set_coords(coord_vars)


start_date = cftime.DatetimeNoLeap(cfg_var_TEMP.year_s, 2, 1)
end_date = cftime.DatetimeNoLeap(cfg_var_TEMP.year_e+1, 1, 1)

In [6]:
import numpy as np
import xarray as xr
from scipy.interpolate import interp1d

def process_coords_thermocline(varname, mask, ds_grid_sub, ds, sd, ed,
                               drop=True, target_temp=20.0,
                               zmax_cm=40000, dz_cm=100,
                               except_coord_vars=None):
    """
    Compute thermocline depth (20 °C isotherm) at each grid cell, then
    take area-weighted mean over the masked region.

    Parameters
    ----------
    varname : str
        Variable name, e.g. 'TEMP'
    mask : xarray.DataArray
        Boolean mask (True = keep)
    ds_grid_sub : xarray.Dataset
        Grid dataset containing 'TAREA' and 'dz'
    ds : xarray.Dataset
        Input dataset containing 3-D variable (ens_LE, time, z_t, nlat, nlon)
    sd, ed : str
        Start and end dates for time selection
    drop : bool, default True
        Drop non-dimensional coordinates
    target_temp : float, default 20.0
        Target temperature (°C)
    zmax_cm : int, default 40000
        Maximum depth (cm) for interpolation
    dz_cm : int, default 100
        Vertical interpolation interval (cm)
    except_coord_vars : list of str, optional
        Coordinate vars to keep
    """
    if except_coord_vars is None:
        except_coord_vars = []

    # ---------------------------------------------
    # Drop unnecessary coordinates and apply mask
    # ---------------------------------------------
    coord_vars = []
    for v in np.array(ds.coords):
        if v not in except_coord_vars:
            coord_vars.append(v)
    for v in np.array(ds.data_vars):
        if v not in except_coord_vars:
            coord_vars.append(v)

    ds = ds.where(mask, drop=True)
    ds = ds.sel(time=slice(sd, ed))

    # TEMP = ds[varname]           # (ens_LE, time, z_t, nlat, nlon)
    TEMP = ds[varname].chunk({"z_t": -1})  # make z_t one chunk per column
    if TEMP.max() > 200:  # probably Kelvin
        TEMP = TEMP - 273.15
    
    z_t = TEMP["z_t"]
    z_new = np.arange(1000, zmax_cm + dz_cm, dz_cm)

    # ---------------------------------------------
    # Function: find 20 °C isotherm depth
    # ---------------------------------------------
    def isotherm_depth_cm(T_profile, z_profile):
        valid = np.isfinite(T_profile)
        if valid.sum() < 2:
            return np.nan

        f = interp1d(z_profile[valid], T_profile[valid],
                     kind="linear", bounds_error=False, fill_value=np.nan)
        T_interp = f(z_new)
        diff = T_interp - target_temp
        sign_change = np.where(np.diff(np.sign(diff)) != 0)[0]
        if len(sign_change) == 0:
            return np.nan

        i = sign_change[0]
        z1, z2 = z_new[i], z_new[i+1]
        T1, T2 = T_interp[i], T_interp[i+1]
        return z1 + (target_temp - T1) * (z2 - z1) / (T2 - T1)

    # # ---------------------------------------------
    # # Apply along the vertical dimension
    # # ---------------------------------------------
    # print("TEMP range:", float(TEMP.min().compute()), float(TEMP.max().compute()))
    # print("z_t range:", float(z_t.min()), float(z_t.max()))
    
    thermo_depth = xr.apply_ufunc(
        isotherm_depth_cm,
        TEMP,
        z_t,
        input_core_dims=[["z_t"], ["z_t"]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float]
    )
    # Result: (ens_LE, time, nlat, nlon)
    thermo_depth = thermo_depth.rename("thermocline_depth_cm")

    # ---------------------------------------------
    # Area-weighted mean over masked region
    # ---------------------------------------------
    area = ds_grid_sub["TAREA"]
    weighted_sum = (thermo_depth * area).sum(dim=["nlat", "nlon"], skipna=True)
    effective_area = area.where(np.isfinite(thermo_depth)).sum(dim=["nlat", "nlon"], skipna=True)
    thermo_depth_mean = weighted_sum / effective_area

    thermo_depth_mean = thermo_depth_mean.rename("thermocline_depth_cm_mean")

    # # ---------------------------------------------
    # # Drop extra coordinates if needed
    # # ---------------------------------------------
    # if drop:
    #     thermo_depth_mean = thermo_depth_mean.drop_vars(coord_vars, errors="ignore")
    # ds["thermocline_depth_cm_mean"] = thermo_depth_mean
    return thermo_depth_mean
    # return ds


In [7]:
cut_grid_info = {
    "lonw": 120,
    "lone": 280,
    "lats": -5,
    "latn": 5
}

ds=ds_grid
mask = ((ds.TLONG >= cut_grid_info["lonw"]) 
        & (ds.TLONG <= cut_grid_info["lone"]) 
        & (ds.TLAT >= cut_grid_info["lats"]) 
        & (ds.TLAT <= cut_grid_info["latn"]))
iy, ix = np.where(mask)

ds_grid_sub = ds_grid.where(mask, drop=True)
area = ds_grid_sub['TAREA']

In [8]:
# # Read LE dataset


# # mask = mask.sel(nlat=ds_grid_sub["nlat"], nlon=ds_grid_sub["nlon"])

# start_time = time.time()

# # TEMP
# dname=cfg_var_TEMP.var
# cfg_var_TEMP.LE_path_load(cfg_var_TEMP.var)
# cfg_var_TEMP.LE_ds = xr.open_mfdataset(cfg_var_TEMP.LE_file_list[0][0:1], 
#                        chunks={'time': 12}, 
#                        combine='nested', 
#                        concat_dim=[[*cfg_var_TEMP.LE_ensembles[0:1]], 'time'], 
#                        parallel=False,
#                        preprocess=lambda ds: process_coords_thermocline(dname, mask, ds_grid_sub, ds, start_date, end_date),
#                        decode_cf=True, 
#                        decode_times=True)      

# cfg_var_TEMP.LE_ds = cfg_var_TEMP.LE_ds.rename({"concat_dim": "ens_LE"})
# new_time = cfg_var_TEMP.LE_ds.time - np.array([datetime.timedelta(days=15)] * len(cfg_var_TEMP.LE_ds.time))
# cfg_var_TEMP.LE_ds = cfg_var_TEMP.LE_ds.assign_coords(time=new_time)
# cfg_var_TEMP.LE_ds= cfg_var_TEMP.LE_ds.compute()

# # WWV writing
# comp_dataset=xr.Dataset()
# comp_dataset['WWV']= cfg_var_TEMP.LE_ds

# filename = (
#     "/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw//XRO_input_LE_WWV"
#     + ".nc"
# )
# if os.path.exists(filename):
#     os.remove(filename)
# comp_dataset.to_netcdf(filename, mode='w')


# end_time = time.time()
# elapsed_time = end_time - start_time
# print('elasped time for reading LE: ' + str(elapsed_time))

In [9]:
import os, re, time, datetime
# Set variable name (e.g., 'TEMP')
dname = cfg_var_TEMP.var
cfg_var_TEMP.LE_path_load(cfg_var_TEMP.var)

# ------------------------------------------------------------
# Loop over all LE ensemble members (typically 50)
# ------------------------------------------------------------
for i in range(len(cfg_var_TEMP.LE_file_list[0])):

    # Ensemble name, e.g., 'LE2-1011.001'
    ens_num = cfg_var_TEMP.LE_ensembles[i]
    ens_pattern = re.compile(r'(LE2-\d{4}\.\d{3})')

    # File list for this member (usually two files per member)
    file_list = cfg_var_TEMP.LE_file_list[0][i]

    # Extract ensemble name from the FIRST file path
    # This ensures correct identification
    match = ens_pattern.search(file_list[0])
    if match:
        ens_name = match.group(1)
    else:
        raise ValueError(f"Cannot extract ensemble name from: {file_list[0]}")
        
    print(f"\n=== Processing ensemble {ens_name} ({i+1}/50) ===")

    start_time = time.time()

    # ------------------------------------------------------------
    # Load dataset for the current ensemble member
    # ------------------------------------------------------------
    ds = xr.open_mfdataset(
        file_list,                      # Choose the file subset (modify if needed)
        chunks={'time': 12},
        combine='nested',
        # concat_dim=[[ens_num], 'time'],     # Create ensemble dimension for this member
        concat_dim=['time'],     # Create ensemble dimension for this member
        parallel=False,
        preprocess=lambda ds: process_coords_thermocline(
            dname, mask, ds_grid_sub, ds, start_date, end_date
        ),
        decode_cf=True,
        decode_times=True
    )

    # Rename concat dimension to a clearer ensemble dimension
    # ds = ds.rename({"concat_dim": "ens_LE"})

    # ------------------------------------------------------------
    # Shift time coordinate by -15 days
    # ------------------------------------------------------------
    new_time = ds.time - np.array([datetime.timedelta(days=15)] * ds.sizes["time"])
    ds = ds.assign_coords(time=new_time)

    # Trigger computation
    ds = ds.compute()

    # ------------------------------------------------------------
    # Wrap into output dataset (WWV or TEMP or other variable)
    # ------------------------------------------------------------
    comp_dataset = xr.Dataset()
    comp_dataset['WWV'] = ds    # Change to 'TEMP' or other variable if needed

    # ------------------------------------------------------------
    # Output file path for this ensemble
    # ------------------------------------------------------------
    out_file = (
        f"/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/"
        f"XRO_input_LE_WWV.{ens_name}.nc"
    )

    # Remove existing file before saving
    if os.path.exists(out_file):
        os.remove(out_file)

    # Save output
    comp_dataset.to_netcdf(out_file, mode='w')

    elapsed = time.time() - start_time
    print(f"Saved: {out_file}")
    print(f"Elapsed time: {elapsed:.2f} sec")

print("\n=== All ensemble members processed successfully! ===")



=== Processing ensemble LE2-1011.001 (1/50) ===
TEMP range: 0.8473646640777588 33.00120544433594
z_t range: 500.0 537500.0
TEMP range: 0.8504199385643005 32.58802032470703
z_t range: 500.0 537500.0
TEMP range: 0.8517112135887146 32.63408660888672
z_t range: 500.0 537500.0
TEMP range: 0.8543847799301147 32.490440368652344
z_t range: 500.0 537500.0
TEMP range: 0.8575409054756165 32.608299255371094
z_t range: 500.0 537500.0
TEMP range: 0.8598198294639587 32.404815673828125
z_t range: 500.0 537500.0
TEMP range: 0.8637115359306335 32.50569534301758
z_t range: 500.0 537500.0
TEMP range: 0.8652224540710449 32.34975051879883
z_t range: 500.0 537500.0
TEMP range: 0.8669857978820801 32.49783706665039
z_t range: 500.0 537500.0
TEMP range: 0.8693515062332153 32.62116241455078
z_t range: 500.0 537500.0
TEMP range: 0.8722156882286072 32.47997283935547
z_t range: 500.0 537500.0
TEMP range: 0.875167191028595 32.51011657714844
z_t range: 500.0 537500.0
TEMP range: 0.8784406781196594 32.83628463745117


In [11]:
## SST indices

cfg_var_SST=CESM2_config()
cfg_var_SST.year_s=1850
cfg_var_SST.year_e=2026
cfg_var_SST.setvar('SST')


In [12]:
# atm_grid read

# fname =( 
#     "/mnt/lustre/proj/earth.system.predictability/ASSM_EXP_timeseries"
# + "/archive/b.e21.BSSP370smbb.f09_g17.assm.projdv7.3_ba-20p5/atm/proc/tseries/month_1/"
# + "b.e21.BSSP370smbb.f09_g17.assm.projdv7.3_ba-20p5.cam.h0.SST.202001-202012.nc"
# )

fname =(
    "/mnt/lustre/proj/earth.system.predictability/ATM_TEST/EXP_FLUX_REF1"
    + "/archive/b.e21.BHISTsmbb.f09_g17.atm_flux.ref1/rest/2015-01-01-00000/"
    + "b.e21.BHISTsmbb.f09_g17.atm_flux.ref1.cam.h0.2014-12.nc"
)

ds_grid_SST=xr.open_mfdataset(fname, decode_cf=True, preprocess=lambda ds: process_coords_sst('SST', ds, start_date, end_date), decode_times=True)

In [13]:
# Read LE dataset

start_time = time.time()

# SST
dname=cfg_var_SST.var
cfg_var_SST.LE_path_load(cfg_var_SST.var)
cfg_var_SST.LE_ds = xr.open_mfdataset(cfg_var_SST.LE_file_list[0][0:1], 
                       chunks={'time': 12}, 
                       combine='nested', 
                       concat_dim=[[*cfg_var_SST.LE_ensembles[0:1]], 'time'], 
                       parallel=False,
                       preprocess=lambda ds: process_coords_sst(dname, ds, start_date, end_date),
                       decode_cf=True, 
                       decode_times=True)      

cfg_var_SST.LE_ds = cfg_var_SST.LE_ds.rename({"concat_dim": "ens_LE"})
new_time = cfg_var_SST.LE_ds.time - np.array([datetime.timedelta(days=15)] * len(cfg_var_SST.LE_ds.time))
cfg_var_SST.LE_ds = cfg_var_SST.LE_ds.assign_coords(time=new_time)
cfg_var_SST.LE_ds= cfg_var_SST.LE_ds.compute()

cfg_var_SST.LE_ds["SST"] = cfg_var_SST.LE_ds["SST"].where(cfg_var_SST.LE_ds["SST"] != 0)
cfg_var_SST.LE_ds["SST"] = cfg_var_SST.LE_ds["SST"]-273.15


# Mask set (ENSO; Nino34)

cut_grid_info = {
    "lonw": 170,
    "lone": 240,
    "lats": -5,
    "latn": 5
}

ds=ds_grid_SST
mask_SST = ((ds.lon >= cut_grid_info["lonw"]) 
        & (ds.lon <= cut_grid_info["lone"]) 
        & (ds.lat >= cut_grid_info["lats"]) 
        & (ds.lat <= cut_grid_info["latn"]))
iy, ix = np.where(mask_SST)

ds_grid_sub = ds_grid_SST.where(mask_SST, drop=True)
ds_grid_sub['AREA']= ds_grid_sub['AREA'].isel(time=0)
area = ds_grid_sub['AREA']
weighted_avg_manual = xr.Dataset()

ds = cfg_var_SST.LE_ds.where(mask_SST, drop=True)
weighted_sum = (ds['SST'] * area).sum(dim=['lat','lon'], skipna=True)
effective_area = area.where(ds['SST'].notnull()).sum(dim=['lat','lon'], skipna=True)
indices = weighted_sum / effective_area

comp_dataset=xr.Dataset()
comp_dataset['NINO34']= indices

filename = (
    "/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw//XRO_input_LE_NINO34"
    + ".nc"
)
if os.path.exists(filename):
    os.remove(filename)
comp_dataset.to_netcdf(filename, mode='w')


end_time = time.time()
elapsed_time = end_time - start_time
print('elasped time for reading LE: ' + str(elapsed_time))

elasped time for reading LE: 15.112976312637329


In [14]:
import os, re, time, datetime
import numpy as np
import xarray as xr

# Set variable name
dname = cfg_var_SST.var
cfg_var_SST.LE_path_load(cfg_var_SST.var)

# Define all index regions
index_regions = {
    "NINO34": {"lonw": 170, "lone": 240, "lats": -5,  "latn": 5},
    "NPMM":   {"lonw": 200, "lone": 240, "lats": 10,  "latn": 25},
    "SPMM":   {"lonw": 250, "lone": 270, "lats": -25, "latn": -15},
    "IOB":    {"lonw": 40,  "lone": 100, "lats": -20, "latn": 20},
    "IOD1":   {"lonw": 50,  "lone": 70,  "lats": -10, "latn": 10},
    "IOD2":   {"lonw": 90,  "lone": 110, "lats": -10, "latn": 0},
    "SIOD1":  {"lonw": 65,  "lone": 85,  "lats": -25, "latn": -10},
    "SIOD2":  {"lonw": 90,  "lone": 120, "lats": -30, "latn": -10},
    "TNA":    {"lonw": 305, "lone": 345, "lats": 5,   "latn": 25},
    "ATL3":   {"lonw": 340, "lone": 360, "lats": -3,  "latn": 3},
    "SASD1":  {"lonw": 300, "lone": 360, "lats": -45, "latn": -35},
    "SASD2":  {"lonw": 320, "lone": 380, "lats": -30, "latn": -20},
}

# Helper: create lat-lon mask (handles wrap-around)
def create_mask(ds, region):
    lon = ds.lon % 360
    lonw, lone = region["lonw"], region["lone"]
    lats, latn = region["lats"], region["latn"]

    if lonw <= lone <= 360:
        mask_lon = (lon >= lonw) & (lon <= lone)
    else:
        mask_lon = (lon >= lonw) | (lon <= (lone % 360))

    mask_lat = (ds.lat >= lats) & (ds.lat <= latn)
    return mask_lon & mask_lat

# Regex for ensemble name
ens_pattern = re.compile(r'(LE2-\d{4}\.\d{3})')

# ------------------------------------------------------------
# Loop over all LE ensemble members
# ------------------------------------------------------------
for i in range(len(cfg_var_SST.LE_file_list[0])):

    file_list = cfg_var_SST.LE_file_list[0][i]

    # Extract ensemble name
    match = ens_pattern.search(file_list[0])
    if match:
        ens_name = match.group(1)
    else:
        raise ValueError(f"Cannot extract ensemble name from: {file_list[0]}")

    print(f"\n=== Processing SST indices for ensemble {ens_name} ({i+1}/50) ===")
    start_time = time.time()

    # ------------------------------------------------------------
    # Load SST dataset (same as WWV structure)
    # ------------------------------------------------------------
    ds = xr.open_mfdataset(
        file_list,
        chunks={'time': 12},
        combine='nested',
        concat_dim=['time'],
        parallel=False,
        preprocess=lambda ds: process_coords_sst(
            dname, ds, start_date, end_date
        ),
        decode_cf=True,
        decode_times=True
    )

    # Time shift
    new_time = ds.time - np.array([datetime.timedelta(days=15)] * ds.sizes["time"])
    ds = ds.assign_coords(time=new_time)
    ds = ds.compute()

    # SST clean
    ds["SST"] = ds["SST"].where(ds["SST"] != 0)
    ds["SST"] = ds["SST"] - 273.15

    # ------------------------------------------------------------
    # Compute all indices
    # ------------------------------------------------------------
    out = xr.Dataset()

    for name, region in index_regions.items():

        # Create mask for this region
        mask = create_mask(ds_grid_SST, region)

        # Subset SST
        ds_reg = ds["SST"].where(mask, drop=True)
        area = ds_grid_SST["AREA"].isel(time=0).where(mask, drop=True)

        weighted_sum = (ds_reg * area).sum(dim=["lat", "lon"], skipna=True)
        effective_area = area.where(ds_reg.notnull()).sum(dim=["lat", "lon"], skipna=True)

        index_ts = weighted_sum / effective_area
        out[name] = index_ts

    # ------------------------------------------------------------
    # Save output per ensemble
    # ------------------------------------------------------------
    out_file = (
        f"/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/"
        f"XRO_input_LE_SST_INDICES.{ens_name}.nc"
    )

    if os.path.exists(out_file):
        os.remove(out_file)

    out.to_netcdf(out_file, mode="w")

    elapsed = time.time() - start_time
    print(f"Saved: {out_file}")
    print(f"Elapsed time: {elapsed:.2f} sec")

print("\n=== All SST indices processed successfully! ===")



=== Processing SST indices for ensemble LE2-1011.001 (1/50) ===
Saved: /mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/XRO_input_LE_SST_INDICES.LE2-1011.001.nc
Elapsed time: 6.69 sec

=== Processing SST indices for ensemble LE2-1031.002 (2/50) ===
Saved: /mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/XRO_input_LE_SST_INDICES.LE2-1031.002.nc
Elapsed time: 6.50 sec

=== Processing SST indices for ensemble LE2-1051.003 (3/50) ===
Saved: /mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/XRO_input_LE_SST_INDICES.LE2-1051.003.nc
Elapsed time: 6.57 sec

=== Processing SST indices for ensemble LE2-1071.004 (4/50) ===
Saved: /mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/XRO_input_LE_SST_INDICES.LE2-1071.004.nc
Elapsed time: 7.04 sec

=== Processing SST indices for ensemble LE2-1091.005 (5/50) ===
Saved: /mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem/XRO_input_LE_SST_INDICES.LE2-1091.005.nc
Elapsed time: 6.54 sec

=== Processing SST indices for ensemble LE2-

In [1]:
import os, re, glob, xarray as xr

# ------------------------------------------------------------
# Paths
# ------------------------------------------------------------
BASE_DIR = "/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO/raw_mem"
OUT_DIR  = "/mnt/lustre/proj/kimyy/Model/CESM2/ESP/for_XRO"

# Regex to extract ensemble names (e.g., LE2-1011.001)
ens_pattern = re.compile(r"(LE2-\d{4}\.\d{3})")

# ------------------------------------------------------------
# Step 1: Merge SST indices + WWV for each ensemble
# ------------------------------------------------------------
sst_files = sorted(glob.glob(f"{BASE_DIR}/XRO_input_LE_SST_INDICES.*.nc"))
merged_files = []   # To store merged file paths
ens_list = []

print("=== Step 1: Merging SST indices with WWV for each ensemble ===")

for sst_file in sst_files:
    match = ens_pattern.search(sst_file)
    if not match:
        continue
    ens_name = match.group(1)
    wwv_file = f"{BASE_DIR}/XRO_input_LE_WWV.{ens_name}.nc"
    out_file = f"{BASE_DIR}/XRO_input_LE_ALL.{ens_name}.nc"

    if not os.path.exists(wwv_file):
        print(f"⚠️ Skipping {ens_name} (WWV file missing)")
        continue

    # Open datasets
    ds_sst = xr.open_dataset(sst_file)
    ds_wwv = xr.open_dataset(wwv_file)

    # Merge SST indices + WWV
    ds_all = xr.merge([ds_sst, ds_wwv], compat="override")

    # Save merged per-member file
    ds_all.to_netcdf(out_file)
    merged_files.append(out_file)
    ens_list.append(ens_name)

    print(f"✅ Merged {ens_name} → {os.path.basename(out_file)}")

print(f"\n--- Step 1 completed: {len(merged_files)} merged files ---\n")

# ------------------------------------------------------------
# Step 2: Combine all merged members into one large dataset
# ------------------------------------------------------------
print("=== Step 2: Combining all members into one dataset ===")

datasets = []
for f in merged_files:
    match = ens_pattern.search(f)
    ens_name = match.group(1)
    ds = xr.open_dataset(f)
    ds = ds.expand_dims({"ens_LE": [ens_name]})  # Add ensemble dimension
    datasets.append(ds)

# Concatenate along ensemble dimension
ds_all = xr.concat(datasets, dim="ens_LE")

# ------------------------------------------------------------
# Optional: Compression for smaller file size
# ------------------------------------------------------------
comp = {v: {"zlib": True, "complevel": 4} for v in ds_all.data_vars}

# Save final file
final_out = f"{OUT_DIR}/XRO_input_LE_ALL_ENSEMBLES.nc"
if os.path.exists(final_out):
    os.remove(final_out)
ds_all.to_netcdf(final_out, encoding=comp)

print(f"\n🎉 All done! Saved merged file for {len(ens_list)} ensembles:")
print(f"👉 {final_out}")


=== Step 1: Merging SST indices with WWV for each ensemble ===
✅ Merged LE2-1011.001 → XRO_input_LE_ALL.LE2-1011.001.nc
✅ Merged LE2-1031.002 → XRO_input_LE_ALL.LE2-1031.002.nc
✅ Merged LE2-1051.003 → XRO_input_LE_ALL.LE2-1051.003.nc
✅ Merged LE2-1071.004 → XRO_input_LE_ALL.LE2-1071.004.nc
✅ Merged LE2-1091.005 → XRO_input_LE_ALL.LE2-1091.005.nc
✅ Merged LE2-1111.006 → XRO_input_LE_ALL.LE2-1111.006.nc
✅ Merged LE2-1131.007 → XRO_input_LE_ALL.LE2-1131.007.nc
✅ Merged LE2-1151.008 → XRO_input_LE_ALL.LE2-1151.008.nc
✅ Merged LE2-1171.009 → XRO_input_LE_ALL.LE2-1171.009.nc
✅ Merged LE2-1191.010 → XRO_input_LE_ALL.LE2-1191.010.nc
✅ Merged LE2-1231.011 → XRO_input_LE_ALL.LE2-1231.011.nc
✅ Merged LE2-1231.012 → XRO_input_LE_ALL.LE2-1231.012.nc
✅ Merged LE2-1231.013 → XRO_input_LE_ALL.LE2-1231.013.nc
✅ Merged LE2-1231.014 → XRO_input_LE_ALL.LE2-1231.014.nc
✅ Merged LE2-1231.015 → XRO_input_LE_ALL.LE2-1231.015.nc
✅ Merged LE2-1231.016 → XRO_input_LE_ALL.LE2-1231.016.nc
✅ Merged LE2-1231.017 → X